##### AI City Builders - AI 수익화의 정석

In [ ]:
# @title Apache License 2.0 라이선스
# 이 코드는 Apache License 2.0에 따라 사용이 허가됩니다.
# 라이선스 전문: https://www.apache.org/licenses/LICENSE-2.0

# 🎙️ AI City Builders: Gemini TTS 시작하기

**AI 수익화의 정석** AI 건물주되기 Day8: 코드로 Text to Speech

---

이 노트북에서는 Gemini 모델의 **TTS(Text-to-Speech)** 기능을 배웁니다.

---
## 🛠️ 설정

### API 키 설정

아래 셀을 실행하려면 API 키가 `GOOGLE_API_KEY`라는 이름의 Colab Secret에 저장되어 있어야 합니다.

In [ ]:
from google.colab import userdata

# Colab Secret에서 API 키 가져오기
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

### SDK 설치 및 초기화

In [ ]:
# 다중 화자 오디오를 위해 1.16 이상 버전 필요
!pip install -U -q "google-genai>=1.16.0"

In [ ]:
from google import genai
from google.genai import types

# Gemini 클라이언트 초기화
client = genai.Client(api_key=GOOGLE_API_KEY)

### 모델 선택

오디오 출력은 `tts` 모델만 지원합니다:
- `gemini-2.5-flash-preview-tts`
- `gemini-2.5-pro-preview-tts`

In [ ]:
# 사용할 TTS 모델 선택
MODEL_ID = "gemini-2.5-flash-preview-tts"

### 도우미 함수 정의

모델에 프롬프트를 전달하고 노트북에서 오디오를 재생하기 위한 헬퍼 함수를 만듭니다.

In [ ]:
# @title 도우미 함수 (이 셀을 실행하세요)

import contextlib
import wave
from IPython.display import Audio

file_index = 0

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    """WAV 파일을 생성하는 컨텍스트 매니저"""
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

def play_audio_blob(blob):
    """오디오 데이터를 파일로 저장하고 재생"""
    global file_index
    file_index += 1

    fname = f'audio_{file_index}.wav'
    with wave_file(fname) as wav:
        wav.writeframes(blob.data)

    return Audio(fname, autoplay=True)

def play_audio(response):
    """API 응답에서 오디오를 추출하여 재생"""
    return play_audio_blob(response.candidates[0].content.parts[0].inline_data)

---
## 🎤 기본 음성 생성

간단한 텍스트를 음성으로 변환해봅시다!

In [ ]:
# 기본 TTS 요청
response = client.models.generate_content(
    model=MODEL_ID,
    contents="다음 문장을 읽어줘: 안녕하세요, 저는 Gemini입니다!",
    config={"response_modalities": ['Audio']},
)

In [ ]:
# 응답에서 오디오 데이터 확인
blob = response.candidates[0].content.parts[0].inline_data
print(f"MIME 타입: {blob.mime_type}")

In [ ]:
# 생성된 오디오 재생
play_audio_blob(blob)

---
## 🎚️ 음성 제어하기

30가지 내장 음성과 24개 지원 언어를 조합하여 다양한 음성을 만들 수 있습니다.

### 음성 선택하기

In [ ]:
# 사용할 음성 선택 (30가지 중 하나 선택)
voice_name = "Sadaltager"

response = client.models.generate_content(
    model=MODEL_ID,
    contents="다음을 읽어줘: 저는 매우 박식한 모델이에요. 5초 기다린 후 '그렇게 생각하지 않으세요?'라고 말해줘.",
    config={
        "response_modalities": ['Audio'],
        "speech_config": {
            "voice_config": {
                "prebuilt_voice_config": {
                    "voice_name": voice_name
                }
            }
        }
    },
)

play_audio(response)

### 언어 변경하기

모델에게 특정 언어로 말하라고 지시하면 됩니다!

**한국어도 지원됩니다! 🇰🇷**

In [ ]:
# 한국어로 읽기
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    다음을 한국어로 읽어주세요:

    안녕하세요! AI City Builders에 오신 것을 환영합니다.
    오늘도 AI와 함께 멋진 하루 보내세요!
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

### 말하는 방식 제어하기

자연어 프롬프트를 사용하여 스타일, 톤, 억양, 속도를 제어할 수 있습니다!

In [ ]:
# 🎃 으스스한 속삭임 스타일
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    으스스한 속삭임으로 다음을 읽어줘:
    "어둠 속에서 무언가가 다가온다...
    뒤를 돌아보지 마세요..."
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

In [ ]:
# ⚡ 빠른 속도로 읽기
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    이 면책조항을 알아들을 수 있는 한 최대한 빠르게 읽어주세요:

    본 사이트의 내용에 대한 오류나 누락에 대해 어떠한 책임도 지지 않습니다.
    이 사이트에 포함된 정보는 완전성, 정확성, 유용성 또는 적시성에 대한 보장 없이 있는 그대로 제공됩니다.
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

---
## 👥 다중 화자

TTS 모델은 2명의 화자가 대화하는 콘텐츠도 생성할 수 있습니다!

In [ ]:
# 두 화자의 대화 생성
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    화자1은 피곤하고 지루한 목소리로, 화자2는 신나고 행복한 목소리로 다음 대화를 읽어줘:

    화자1: 그래서... 오늘 일정이 뭐야?
    화자2: 절대 못 맞힐걸!
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

### 화자별 음성 지정하기

각 참여자의 음성을 선택하고 이름을 지정할 수 있습니다.

먼저 두 AI 전문가의 대화 스크립트를 생성해봅시다:

In [ ]:
# Gemini로 대화 스크립트 생성
transcript = client.models.generate_content(
    model='gemini-2.5-flash',
    contents="""
      안녕하세요! AI 전문가인 제이 멘토와 그의 조수 민지의 팟캐스트에서
      발췌한 것처럼 들리는 짧은 대화(약 100단어)를 생성해주세요.
      주제는 'AI로 수익 창출하기'입니다.
      한국어로 작성해주세요.
    """
).text

print("📝 생성된 대화 스크립트:")
print(transcript)

In [ ]:
# 다중 화자 설정 구성
config = types.GenerateContentConfig(
    response_modalities=["AUDIO"],
    speech_config=types.SpeechConfig(
        multi_speaker_voice_config=types.MultiSpeakerVoiceConfig(
            speaker_voice_configs=[
                # 제이 멘토 음성 설정
                types.SpeakerVoiceConfig(
                    speaker='제이 멘토',
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Orus',
                        )
                    )
                ),
                # 민지 음성 설정
                types.SpeakerVoiceConfig(
                    speaker='민지',
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Leda',
                        )
                    )
                ),
            ]
        )
    )
)

# 대화 TTS 생성
response = client.models.generate_content(
    model=MODEL_ID,
    contents="다음 제이 멘토와 민지의 대화를 읽어줘: " + transcript,
    config=config,
)

play_audio(response)

---

🎉 **축하합니다!** Gemini TTS 실습을 완료했습니다!